# An Improved Symetric Homomorphic Encryption Scheme

In this notebook, a symetric homomorphic encryption system is introduced that uses a sequence of singular linear transforms to make known plaintext attacks impossible, and give a cipher text of a different, randomely chosen size to the plaintext.

In [1]:
import numpy as np

In [2]:
message = np.array([9.0, 4.0, 2.0])
key1 = np.random.randint(1, 30, ((len(message) + np.random.randint(2, 6)), len(message)))
key1

array([[ 5, 23, 24],
       [ 7, 25,  5],
       [18, 25, 20],
       [22, 17, 29],
       [28,  6,  9],
       [12, 18, 24],
       [21,  9,  1]])

In [3]:
# know m is always > n
key1inv = np.matmul((np.linalg.inv(np.matmul(key1.T, key1))), key1.T)
np.matmul(key1inv, key1)

array([[ 1.00000000e+00, -8.32667268e-17,  5.89805982e-17],
       [ 1.38777878e-16,  1.00000000e+00,  1.63064007e-16],
       [-5.55111512e-17, -2.77555756e-17,  1.00000000e+00]])

In [4]:
key2 = np.random.randint(1, 30, ((np.random.randint(1, 4) + key1.shape[0]), key1.shape[0]))
# check
key2

array([[23, 19,  3, 28, 27, 24, 14],
       [ 5, 11, 16, 22,  6, 25, 29],
       [16, 13,  6, 13, 28, 11, 20],
       [29, 27, 24,  4, 13,  9,  5],
       [ 4, 19, 11,  5, 26, 27, 14],
       [26, 21, 16, 19, 27, 17, 11],
       [17, 14,  4, 10, 11, 17, 14],
       [29, 19,  5, 21, 19,  8, 12]])

In [5]:
# know always m>n check to see if produces I
key2inv = np.matmul((np.linalg.inv(np.matmul(key2.T, key2))), key2.T)
np.matmul(key2inv, key2)

array([[ 1.00000000e+00, -1.08801856e-14, -6.71684930e-15,
        -1.42108547e-14, -2.62012634e-14, -2.44249065e-14,
        -1.77635684e-14],
       [ 2.66453526e-14,  1.00000000e+00,  1.34336986e-14,
         3.46389584e-14,  5.24025268e-14,  4.39648318e-14,
         3.28626015e-14],
       [-1.50990331e-14, -1.39888101e-14,  1.00000000e+00,
        -1.19904087e-14, -1.97619698e-14, -1.36002321e-14,
        -1.05471187e-14],
       [ 7.54951657e-15,  5.99520433e-15,  4.02455846e-15,
         1.00000000e+00,  1.07691633e-14,  8.32667268e-15,
         6.43929354e-15],
       [-1.38777878e-15, -1.58206781e-15,  1.94289029e-16,
        -2.16493490e-15,  1.00000000e+00, -1.94289029e-15,
        -1.58206781e-15],
       [-2.19824159e-14, -1.80966353e-14, -1.22124533e-14,
        -1.56541446e-14, -2.62012634e-14,  1.00000000e+00,
        -1.47659662e-14],
       [ 7.77156117e-16,  9.43689571e-16,  4.44089210e-16,
         1.30451205e-15,  3.10862447e-15,  1.66533454e-15,
         1.0000000

In [6]:
# check!
def encrypt(message, key1, key2):
    # scale up plaintext by a whole bunch to avoid noise damage and add random noise vector
    message += (np.random.rand(len(message)) / 2**20)
    message *= 40
    
    ciphertext = np.matmul(key1, message)
    ciphertext = (2 * ciphertext) + 19
    ciphertext = np.matmul(key2, ciphertext)
    return ciphertext

ciphertext = encrypt(message, key1, key2)
ciphertext *= 6
print("The ciphertext is: ", ciphertext)

The ciphertext is:  [16387574.02140881 13853797.6740994  12738439.50726764 12295375.49072172
 12466645.47017003 16211779.97928754  9755359.19007522 13020883.59932926]


In [7]:
# define a rounding function for use in making sure everything ends up ok
def myround(a, decimals=0):
     return np.around(a-10**(-(decimals+5)), decimals=decimals)
    
# make decryption function, just inverse order of encryption function
def decrypt(ciphertext, key1, key2):
    key1inv = np.matmul((np.linalg.inv(np.matmul(key1.T, key1))), key1.T)
    key2inv = np.matmul((np.linalg.inv(np.matmul(key2.T, key2))), key2.T)
    
    plaintext = np.matmul(ciphertext, key2inv.T)
    plaintext = (0.5 * plaintext) - 19
    plaintext = np.matmul(plaintext, key1inv.T)
    plaintext *= (1/40)
    return plaintext

In [9]:
# now make sure everything turns out ok
plaintext = decrypt(ciphertext, key1, key2)
plaintext = myround(plaintext)
print("The plaintext is \n", plaintext, " \nAfter has been scaled up by 6 in ciphertext step")

The plaintext is 
 [54. 24. 12.]  
After has been scaled up by 6 in ciphertext step


### Results
have the outlines functions live above, and the tests live below.

Note how the plaintext needs to be scaled up and down before and after the operations as to avoid damage from the noise.

## Bibliography
* Efficient Integer Vector Homomorphic Encryption Paper
* Some other Papers